# Generating dataset
---
---

# Imports

#### Standard library imports

In [ ]:
import sys
import pickle
import os

sys.path.append("../../")

#### Third party imports

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 500)

import numpy as np

#### Local application imports

In [ ]:
%load_ext autoreload
%autoreload 2

from pkg_dir.config import *
from pkg_dir.src.utils import *
from pkg_dir.src.functions import *
from pkg_dir.src.parameters import *

# Base dataset with no metadata

### Query on database

In [ ]:
## Function parameters
db_crds = 'pc_db_prod'
query = '''
SELECT cita.citaid as appointment_id,
       citafecha as appointment_date,
       citahorad as appointment_start_time,
       citahorah as appointment_end_time,
       citaestado as appointment_status,
       citasubsecuente as recurring_patient,
       u.usuarionomfull as doctor,
       e.especialidadnom as medical_specialty,
       su.sucursalnom as clinic,
       se.servicionom as service,
       p.pacienteid as patient_id,
       p.pacientesexo as patient_sex,
       p.pacientefnac as patient_birth_date

FROM cita

 INNER JOIN usuario u ON cita.citadoctorid = u.usuarioid
 INNER JOIN servicio se ON cita.servicioid = se.servicioid
 INNER JOIN especialidad e ON u.usuarioespecialidadid = e.especialidadid
 INNER JOIN sucursal su ON cita.citasucursalid = su.sucursalid
 LEFT JOIN paciente p ON cita.pacienteid = p.pacienteid

WHERE citafecha >= '2022-01-01'
  AND citafecha <= '2023-12-31'

;
'''


In [ ]:
dfx = sql_string_to_df(db_crds, query)
dfx.columns = [
    'appointment_id', 
    'appointment_date', 
    'appointment_start_time', 
    'appointment_end_time', 
    'appointment_status', 
    'recurring_patient', 
    'doctor', 
    'medical_specialty', 
    'clinic', 
    'service', 
    'patient_id', 
    'patient_sex', 
    'patient_birth_date'
]

### Formatting date columns

In [ ]:
## Formatting appointment dates
dfx['appointment_start'] = dfx['appointment_date'].astype('str') + ' ' + dfx['appointment_start_time']
dfx['appointment_start'] = pd.to_datetime(dfx['appointment_start'], format='%Y-%m-%d %H:%M')

dfx['appointment_end'] = dfx['appointment_date'].astype('str') + ' ' + dfx['appointment_end_time']
dfx['appointment_end'] = pd.to_datetime(dfx['appointment_end'], format='%Y-%m-%d %H:%M', errors='coerce')

In [ ]:
dfx = dfx[~dfx['appointment_end'].isna()].copy()

In [ ]:
## Appointment weekday
dfx['appointment_weekday'] = dfx['appointment_start'].dt.weekday

In [ ]:
## Appointment duration
dfx['appointment_dur_min'] = dfx['appointment_end'] - dfx['appointment_start']
dfx['appointment_dur_min'] = dfx['appointment_dur_min'].dt.total_seconds() / 60

### Age of patients

In [ ]:
## Numerical age
dfx['patient_age'] = dfx['patient_birth_date'].fillna('2025')
dfx['patient_age'] = dfx['patient_age'].apply(lambda x: str(x)[:4])
dfx['patient_age'] = 2024 - dfx['patient_age'].astype('float')
dfx['patient_age'] = dfx['patient_age'].apply(lambda x: np.nan if (x <= 0 or x >=110) else x)

### Patient sex clean up

In [ ]:
dfx['patient_sex'] = dfx['patient_sex'].replace({' ': None})

In [ ]:
dfx['patient_sex2'].unique()

### Recurring patient field

In [ ]:
dfx['recurring_patient'] = dfx['recurring_patient'].replace({' ': None})

In [ ]:
dfx['recurring_patient'] = dfx['recurring_patient'].map(
    {
        'P': 'first',
        'S': 'recurrent',
    }
)

In [ ]:
dfx['recurring_patient'].unique()

### Appointment status simplification

In [ ]:
## Reference to simplify labels in appointment status
values_map = {

    ## Completed
    'COMPLETADA': 'completed',
    'SALA_ESPERA': 'completed',
    'CONSULTA': 'completed',
    'CONFIRMADO_PAGO': 'completed',

    ## No shows
    'NO_PRESENTO': 'no_show',
    'MENSAJE_DEJADO': 'no_show',
    'NO_CONFIRMADO': 'no_show',
    'CONFIRMADO': 'no_show',
    'LLEGA_TARDE_ESPERA': 'no_show',
    'REAGENDAR': 'no_show',
    'TRIAGE': 'no_show',
    'TRIAGE_COMPLETO': 'no_show',
    'LISTA_ESPERA': 'no_show',
    'VALIDA_DATOS': 'no_show',

    ## Unused
    'DISPONIBLE': 'unused',

    ## Blocked
    'BLOQUEADO': 'blocked',

    ## Canceled
    'CANCELA_PACIENTE': 'cancel_patient',
    'CANCELA_EMPLEADO': 'cancel_employee',
    
    
}

In [ ]:
## Mapping values according to reference
dfx['appointment_status_simplified'] = dfx['appointment_status'].map(values_map).fillna(dfx['appointment_status'])

### Formatting data

In [ ]:
dfx.columns.tolist()

In [ ]:
## Reordering columns
rc = [
 ## Appointment features
 'appointment_id',
 'appointment_weekday',
 'appointment_start',
 'appointment_end',
 'appointment_dur_min',
 'recurring_patient',
 'appointment_status_simplified',
 'clinic',
 ## Patient info
 'patient_age',
 'patient_sex',
 ## Medical info
 'doctor',
 'medical_specialty',
]

In [ ]:
dfx = dfx.loc[:, rc].copy()

### Saving/loading data

In [ ]:
## Saving df as pickle and storing it locally
path = '../../pkg_dir/data/pickles/robs'
name = 'appts_extract.pkl'

pickle.dump(
    dfx,
    open(
        os.path.join(path, name),
        'wb'
    )
)

In [ ]:
## Params
path = '../../pkg_dir/data/pickles/robs'
name = 'appts_extract.pkl'


## Reading extract object saved as pickle locally
pkl_obj = path + "/" + name

with open(pkl_obj, 'rb') as obj_content:
    dfx = pickle.load(obj_content)

In [ ]:
## Saving as csv
dataset.to_csv('../../pkg_dir/data/pickles/robs/dataset.csv')

# Metadata actions

### Extracting from SQL

In [ ]:
sql_string = """
SELECT cita.citaid as appointment_id,
       c.cestatusobs as metadata,
       SUBSTRING(c.cestatusobs, '^[^ ]*') as metadata_usr,
       SUBSTRING(c.cestatusobs, '\((.+?)\)') as metadata_datetime,
       SUBSTRING(c.cestatusobs, ' :(.*?)  ') as metadata_action,
       SUBSTRING(c.cestatusobs, ' :(.*)') as metadata_action_full

FROM cita

 INNER JOIN citaestatus c ON cita.citaid = c.citaid AND cita.citaanio = c.citaanio


WHERE citafecha >= '2022-01-01'
  AND citafecha <= '2023-12-31'

;
"""

In [ ]:
dfx = sql_string_to_df('pc_db_prod', sql_string)

In [ ]:
dfx

In [ ]:
dfx.rename(
    columns={
        0: "appointment_id",
        1: "metadata_full",
        2: "user",
        3: "date",
        4: "action",
        5: "action_full",
    }, 
    inplace=True
)

### Pickle save/load

In [ ]:
## Saving df as pickle and storing it locally
path = '../../pkg_dir/data/pickles/robs'
name = 'metadata_raw.pkl'

pickle.dump(
    dfx,
    open(
        os.path.join(path, name),
        'wb'
    )
)

In [ ]:
## Saving df as pickle and storing it locally
path = '../../pkg_dir/data/pickles/robs'
name = 'metadata_raw.pkl'


## Reading extract object saved as pickle locally
pkl_obj = path + "/" + name

with open(pkl_obj, 'rb') as obj_content:
    dfm = pickle.load(obj_content)

### Cleaning actions fields

In [ ]:
dfz = dfx.copy()

In [ ]:
## Dummy col
dfz['action_clean'] = 'irrelevant'

In [ ]:
### Confirmado
mr1 = dfz['action_full'].str.contains('Confirmado')

dfz.loc[mr1, 'action_clean'] = 'confirmed'

In [ ]:
### Creado
mr1 = dfz['action_full'].str.contains('Creado')

dfz.loc[mr1, 'action_clean'] = 'created'

In [ ]:
### Sala espera
mr1 = dfz['action_full'].str.contains('En sala de espera')

dfz.loc[mr1, 'action_clean'] = 'in_waiting_room'

In [ ]:
### Completado
mr1 = dfz['action_full'].str.contains('Completado')
mr2 = dfz['action_full'].str.contains('Completada')
mrs = mr1 | mr2

dfz.loc[mrs, 'action_clean'] = 'completed'

In [ ]:
### En consulta
mr1 = dfz['action_full'].str.contains('En consulta')

dfz.loc[mr1, 'action_clean'] = 'with_doctor'

In [ ]:
### No se presentó
mr1 = dfz['action_full'].str.contains('No se presentó')

dfz.loc[mr1, 'action_clean'] = 'no_show'

In [ ]:
### Cancelada por el paciente
mr1 = dfz['action_full'].str.contains('Cancelada por el paciente')

dfz.loc[mr1, 'action_clean'] = 'canceled_by_patient'

In [ ]:
### Cancelada por empledo
mr1 = dfz['action_full'].str.contains('Cancelada por empledo')

dfz.loc[mr1, 'action_clean'] = 'canceled_by_employee'

In [ ]:
### Se actualiza la fecha/hora
mr1 = dfz['action_full'].str.contains('Se actualiza la fecha/hora')

dfz.loc[mr1, 'action_clean'] = 'update_date_hour'

In [ ]:
### Disponible
mr1 = dfz['action_full'].str.contains('Disponible')

dfz.loc[mr1, 'action_clean'] = 'available'

In [ ]:
### Mensaje dejado
mr1 = dfz['action_full'].str.contains('Mensaje dejado')

dfz.loc[mr1, 'action_clean'] = 'message_left'

### Formatting and adjusting

In [ ]:
dfz[dfz['action_clean'] == 'completed']

In [ ]:
## Dropping irrelevant columns
mr1 = dfz['action_clean'] != 'irrelevant'
rc = ['appointment_id', 'user', 'date', 'action_clean']

dfz = dfz.loc[mr1, rc].copy()

In [ ]:
## Adjusting dateformat
dfz['date'] = pd.to_datetime(dfz['date'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

dfz = dfz.loc[~dfz['date'].isna(), :].copy()

In [ ]:
## Sorting values 
dfz.sort_values(by=['appointment_id', 'action_clean', 'date'], inplace=True, ascending=False)

In [ ]:
## Keeping latest 
dfz.drop_duplicates(keep='first', subset=['appointment_id', 'action_clean'], inplace=True)

### Pivot

In [ ]:
dfp = dfz.pivot(index='appointment_id', columns='action_clean', values=['user', 'date'])

In [ ]:
dfp.columns = ['meta__' + col[1] + '_$' + col[0] for col in dfp.columns]

In [ ]:
cols = dfp.columns.tolist()
cols.sort()

dfp = dfp.loc[:, cols].copy()

### Pickle save/load

In [ ]:
## Saving df as pickle and storing it locally
path = '../../pkg_dir/data/pickles/robs'
name = 'pivot_df_v2.pkl'

pickle.dump(
    dfp,
    open(
        os.path.join(path, name),
        'wb'
    )
)

In [ ]:
## Params
path = '../../pkg_dir/data/pickles/robs'
name = 'pivot_df_v2.pkl'


## Reading extract object saved as pickle locally
pkl_obj = path + "/" + name

with open(pkl_obj, 'rb') as obj_content:
    dfp = pickle.load(obj_content)

# Merging metadata + new features

### Merging

In [ ]:
dfp.reset_index(inplace = True)

In [ ]:
dataset = pd.merge(dfx, dfp, on='appointment_id', how='left')

### Creation to appointment start

In [ ]:
## Appointment duration
dataset['creation_to_start_hrs'] = dataset['appointment_start'] - dataset['meta__created_$date']
dataset['creation_to_start_hrs'] = dataset['creation_to_start_hrs'].dt.total_seconds() / (60*60)
dataset['creation_to_start_hrs'] = dataset['creation_to_start_hrs'].round(2)

### Confirmation to appointment start

In [ ]:
## Appointment duration
dataset['confirm_to_start_hrs'] = dataset['appointment_start'] - dataset['meta__confirmed_$date']
dataset['confirm_to_start_hrs'] = dataset['confirm_to_start_hrs'].dt.total_seconds() / (60*60)
dataset['confirm_to_start_hrs'] = dataset['confirm_to_start_hrs'].round(2)

In [ ]:
## Confirmation tag
dataset['patient_confirm'] = 0
dataset.loc[dataset['confirm_to_start_hrs'].notnull(), 'patient_confirm'] = 1

### Source of appointment

In [ ]:
## Appointment duration
dataset['appointment_source'] = dataset['meta__created_$user'].apply(lambda x: 'online' if x == 'citas.online' else 'phone')

### Appointment moved

In [ ]:
## Appointment duration
dataset['appointment_date_update'] = 'no_update'
dataset.loc[dataset['meta__update_date_hour_$user'].notnull(), 'appointment_date_update'] = 'update'

### Confirmation message sent

In [ ]:
## message for confirmation
dataset['confirm_request_msg'] = 0
dataset.loc[dataset['meta__message_left_$user'].notnull(), 'confirm_request_msg'] = 1

In [ ]:
dataset['confirm_request_msg'].value_counts()

### Select relevant columns

In [ ]:
dataset.columns.tolist()

In [ ]:
## Reordering columns
rc = [
 ## Appointment features
 'appointment_id',
 'appointment_weekday',
 'appointment_start',
 'appointment_dur_min',
 'recurring_patient',
 'creation_to_start_hrs',
 'confirm_request_msg',
 'patient_confirm',
 'confirm_to_start_hrs',
 'clinic',
 'appointment_source',
 'appointment_date_update',
 ## Patient info
 'patient_age',
 'patient_sex',
 ## Medical info
 'doctor',
 'medical_specialty',
 ## Label
 'appointment_status_simplified',
]

In [ ]:
dataset = dataset.loc[:, rc].copy()

---
# Patient history

In [ ]:
## Params
path = '../../pkg_dir/data/pickles/robs'
name = 'appts_extract.pkl'


## Reading extract object saved as pickle locally
pkl_obj = path + "/" + name

with open(pkl_obj, 'rb') as obj_content:
    dfx = pickle.load(obj_content)

### Relevant data

In [ ]:
dfx['appointment_status_simplified'].unique()

In [ ]:
mr1 = dfx['appointment_status_simplified'].isin(
    [
        'completed',
        'no_show'
    ]
)
mr2 = dfx['patient_id'].notnull()
mrs = mr1 & mr2

rc = [
    'appointment_id',
    'patient_id',
    'appointment_status_simplified',
]

### Grouping

In [ ]:
dfx

---
# Final dataset

In [ ]:
## Saving df as pickle and storing it locally
path = '../../pkg_dir/data/pickles/robs'
name = 'dataset.pkl'

pickle.dump(
    dataset,
    open(
        os.path.join(path, name),
        'wb'
    )
)

In [ ]:
## Params
path = '../../pkg_dir/data/pickles/robs'
name = 'dataset.pkl'


## Reading extract object saved as pickle locally
pkl_obj = path + "/" + name

with open(pkl_obj, 'rb') as obj_content:
    dataset = pickle.load(obj_content)

---
# Subdatasets

### v1: confirmed only; completed and no_shows

In [ ]:
mr1 = dataset['patient_confirm'] == 1
dataset.groupby(
    [
        pd.Grouper(key='appointment_start', freq='Y'),
        'appointment_status_simplified',
        'patient_confirm',
    ]
).agg(
    {
        'appointment_id': 'count'
    }
)

In [ ]:
mr2 = dataset['appointment_status_simplified'].isin(
    [
        'cancel_employee',
        'cancel_patient',
        'completed',
        'no_show',
    ]
)

mrs = mr2

dv1 = dataset.loc[mrs, :].copy()

In [ ]:
dv1['appointment_status_simplified'] = dv1['appointment_status_simplified'].map(
    {
        'cancel_employee': 'cancel',
        'cancel_patient': 'cancel',
    }
).fillna(dv1['appointment_status_simplified'])

In [ ]:
## Saving df as pickle and storing it locally
path = '../../pkg_dir/data/pickles/robs'
name = 'dataset_v1.pkl'

pickle.dump(
    dv1,
    open(
        os.path.join(path, name),
        'wb'
    )
)

In [ ]:
## Params
path = '../../pkg_dir/data/pickles/robs'
name = 'dataset_v1.pkl'


## Reading extract object saved as pickle locally
pkl_obj = path + "/" + name

with open(pkl_obj, 'rb') as obj_content:
    dv1 = pickle.load(obj_content)

### v2: completed and no_shows

In [ ]:
dv1 = dataset.copy()

In [ ]:
mr1 = dataset['appointment_status_simplified'].isin(['cancel_employee', 'cancel_patient', 'completed', 'no_show'])
mr2 = dataset['']

In [ ]:
dataset['appointment_status_simplified'] = dataset['appointment_status_simplified'].map({'cancel_employee': 'cancel', 'cancel_patient': 'cancel'}).fillna(dataset['appointment_status_simplified'])
dataset['appointment_status_simplified'].map

---
# Notes

### Patients first time vs second

In [ ]:
dfx.groupby(
    [
        'recurring_patient'
    ]
).agg(
    {
        'appointment_id': 'count'
    }
)

---

---